In [22]:
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit import Aer
from qiskit import ClassicalRegister,  transpile
from qiskit.visualization import plot_histogram
import numpy as np
from qiskit import BasicAer
from qiskit import BasicAer, execute
import statistics
import qiskit
from qiskit import assemble, transpile
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter

In [4]:
theta = [np.pi/8, np.pi/4, np.pi/2, -np.pi/4, np.pi/2, np.pi, np.pi/2, np.pi/2]
qr = QuantumRegister(7,'a')
cr = ClassicalRegister(1,'creg')
qc = QuantumCircuit(qr,cr)
num_shots = 1000
for i in range(0,7):
    qc.ry(theta[i],qr[i])
for i in range(0,6):
    qc.cu(theta[i],0,0,0, qr[i], qr[6])
qc.measure(6,0)
job = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=num_shots)
hist = job.result().get_counts()
print(hist)
#Analysis of the output for multiple runs
prob_1a = []
for expt in range(1,10):
    job = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=num_shots)
    prob_1a.append(job.result().get_counts()['1']/num_shots)
print(prob_1a)

{'0': 738, '1': 262}
[0.278, 0.253, 0.276, 0.272, 0.273, 0.265, 0.267, 0.281, 0.272]


In [33]:
#extra
from qiskit.providers.aer import QasmSimulator
theta = [np.pi/8, np.pi/4, np.pi/2, -np.pi/4, np.pi/2, np.pi, np.pi/2, np.pi/2]
qr = QuantumRegister(7,'a')
cr = ClassicalRegister(1,'creg')
qc = QuantumCircuit(qr,cr)
num_shots = 1000
for i in range(0,7):
    qc.ry(theta[i],qr[i])
for i in range(0,6):
    qc.cu(theta[i],0,0,0, qr[i], qr[6])
qc.measure(6,0)
qc.save_statevector()
backend = QasmSimulator()
backend_options = {'method': 'statevector'}
job = execute(qc, backend, backend_options=backend_options)
job_result_1 = job.result().get_statevector(qc)
qst_1 = state_tomography_circuits(qc, qr[6])
print(qst_1)

[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd0e80db910>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd0e80db700>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd0f8c00c10>]


In [43]:
p1Q = 0.001
p2Q = 0.01
theta = [np.pi/8, np.pi/4, np.pi/2, -np.pi/4, np.pi/2, np.pi, np.pi/2, np.pi/2]
qr = QuantumRegister(7,'a')
cr = ClassicalRegister(1,'creg')
qc = QuantumCircuit(qr,cr)
for i in range(0,7):
    qc.ry(theta[i],qr[i])
for i in range(0,6):
    qc.cu(theta[i],0,0,0, qr[i], qr[6])
qc.measure(6,0)
error_1 = noise.depolarizing_error(p1Q, 1)
error_2 = noise.depolarizing_error(p2Q, 2)
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['x','z','h','ry'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx','cu'])
error_gate1 = pauli_error([('X',p2Q), ('I', 1 - p2Q)])
error_gate2 = error_gate1.tensor(error_gate1)
error_reset = pauli_error([('X', p1Q), ('I', 1 - p1Q)])
error_meas = pauli_error([('X',p1Q), ('I', 1 - p1Q)])
noise_model.add_all_qubit_quantum_error(error_reset, "reset")
noise_model.add_all_qubit_quantum_error(error_meas, "measure")
basis_gates = noise_model.basis_gates
prob_1a = []
for expt in range(1,10):
    job = execute(qc, Aer.get_backend('qasm_simulator'),
                     basis_gates=basis_gates,
                     noise_model=noise_model, shots=num_shots)
    prob_1a.append(job.result().get_counts()['1']/num_shots)
print(prob_1a)

[0.294, 0.272, 0.281, 0.277, 0.272, 0.3, 0.284, 0.281, 0.287]


In [26]:
def nonlocal_gate(qc, ctrl_qubit, target_qubit, comm_1, comm_2, theta):
    qc.h(comm_1)
    qc.cx(comm_1,comm_2)
    qc.cx(ctrl_qubit,comm_1)
    qc.measure(comm_1,0)
    qc.x(comm_2).c_if(cr[0], 1)
    qc.cu(theta,0,0,0,comm_2, target_qubit)
    qc.h(comm_2)
    qc.measure(comm_2,1)
    qc.z(ctrl_qubit).c_if(cr[1], 1)
    qc.reset(qr[7])
    qc.reset(qr[8])
    return qc

qr = QuantumRegister(9,'a')
cr = ClassicalRegister(2,'creg')
qc = QuantumCircuit(qr,cr)

theta = [np.pi/8, np.pi/4, np.pi/2, -np.pi/4, np.pi/2, np.pi, np.pi/2, np.pi/2]
for i in range(0,7):
    qc.ry(theta[i],qr[i])
for i in range(0,6):
    nonlocal_gate(qc, qr[i], qr[6], qr[7], qr[8],theta[i])
qc.measure(6,0)
prob_2a = []
for expt in range(1,10):
    job = execute(qc, BasicAer.get_backend('qasm_simulator'), shots=num_shots) 
    hist = job.result().get_counts()
    a_0 = []
    a_1 = []
    key = list(hist.keys())
    val = list(hist.values())
    for i in range(len(key)):
        if '1' in key[i][1]:
            a_1.append(val[i])
        elif '0' in key[i][1]:
            a_0.append(val[i])
    hist={}
    hist['0'] = sum(a_0)
    hist['1'] = sum(a_1) 
    prob_2a.append(hist['1']/num_shots)
print(prob_2a)

[0.281, 0.292, 0.259, 0.265, 0.277, 0.285, 0.275, 0.278, 0.244]


In [27]:
p1Q = 0.001 #single qubit gate error
p2Q = 0.01 #two qubit gate error
def nonlocal_gate(qc, ctrl_qubit, target_qubit, comm_1, comm_2, theta):
    qc.h(comm_1)
    qc.cx(comm_1,comm_2)
    qc.cx(ctrl_qubit,comm_1)
    qc.measure(comm_1,0)
    qc.x(comm_2).c_if(cr[0], 1)
    qc.cu(theta,0,0,0,comm_2, target_qubit)
    qc.h(comm_2)
    qc.measure(comm_2,1)
    qc.z(ctrl_qubit).c_if(cr[1], 1)
    qc.reset(qr[7])
    qc.reset(qr[8])
    return qc

qr = QuantumRegister(9,'a')
cr = ClassicalRegister(2,'creg')
qc = QuantumCircuit(qr,cr)

theta = [np.pi/8, np.pi/4, np.pi/2, -np.pi/4, np.pi/2, np.pi, np.pi/2, np.pi/2]
for i in range(0,7):
    qc.ry(theta[i],qr[i])
for i in range(0,6):
    nonlocal_gate(qc, qr[i], qr[6], qr[7], qr[8],theta[i])
qc.measure(6,0)
error_1 = noise.depolarizing_error(p1Q, 1)
error_2 = noise.depolarizing_error(p2Q, 2)
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['x','z','h','ry'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx','cu'])
error_gate1 = pauli_error([('X',p2Q), ('I', 1 - p2Q)])
error_gate2 = error_gate1.tensor(error_gate1)
error_reset = pauli_error([('X', p1Q), ('I', 1 - p1Q)])
error_meas = pauli_error([('X',p1Q), ('I', 1 - p1Q)])
noise_model.add_all_qubit_quantum_error(error_reset, "reset")
noise_model.add_all_qubit_quantum_error(error_meas, "measure")
basis_gates = noise_model.basis_gates

var= []
prob_2b = []
for expt in range(1,10):
    result = execute(qc, Aer.get_backend('qasm_simulator'),
                     basis_gates=basis_gates,
                     noise_model=noise_model, shots=num_shots).result().get_counts(0)
    a_0 = []
    a_1 = []
    key = list(result.keys())
    val = list(result.values())
    for i in range(len(key)):
        if '1' in key[i][1]:
            a_1.append(val[i])
        elif '0' in key[i][1]:
            a_0.append(val[i])
    hist={}
    hist['0'] = sum(a_0)
    hist['1'] = sum(a_1) 
    prob_2b.append(hist['1']/num_shots)
print(prob_2b)

[0.272, 0.295, 0.286, 0.295, 0.294, 0.27, 0.31, 0.287, 0.302]


In [28]:
print(statistics.variance(np.subtract(prob_1a,prob_1b)), statistics.variance(np.subtract(prob_1a,prob_2a)), statistics.variance(np.subtract(prob_1a,prob_2b)))

0.0003717499999999993 0.00012377777777777788 0.0003321944444444437
